In [1]:
!pip install nltk==3.4.5
!pip install torch
!pip install transformers

  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449923 sha256=32a4066dfaa6e6562451832e9d3e05cc21c4c89df580af1fb0d5446d09b42257
  Stored in directory: c:\users\samad\appdata\local\pip\cache\wheels\04\32\57\69e42ad50941013def31e288c6e06bb569442dd993a123cb76
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
#for pre processing
import csv,numpy 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Approach 1

In [3]:


# column index of the column we want to read
column_index = 2

# list to store the column values
column_values = []

# open the CSV file in read mode
'''
with open('/content/drive/MyDrive/DevRev/train_data.csv', 'r') as csv_file:
    # creating a CSV reader object
    reader = csv.reader(csv_file)
    
    # iterating over the rows in the CSV file
    for row in reader:
        # appending the value at the specified column index to the list
        column_values.append(row[column_index])
'''
path = 'train_data.csv'
data = pd.read_csv(path)
for para in data['Paragraph']:
  column_values.append(para)

# column_values now contains the values from the specified column

# list of paragraphs
paragraphs = column_values

# question
question = "What is the purpose of preprocessing text data?"

# lowercase the paragraphs and question
paragraphs = [p.lower() for p in paragraphs]
question = question.lower()

# removing stop words and punctuation from the paragraphs and question
stop_words = set(stopwords.words('english'))
punctuation = set(['.', ',', ':', ';', '?'])

filtered_paragraphs = []
for p in paragraphs:
    filtered_p = []
    for w in word_tokenize(p):
        if w not in stop_words and w not in punctuation:
            filtered_p.append(w)
    filtered_paragraphs.append(filtered_p)

filtered_question = []
for w in word_tokenize(question):
    if w not in stop_words and w not in punctuation:
        filtered_question.append(w)

# stem the paragraphs and question
stemmer = PorterStemmer()

stemmed_paragraphs = []
for p in filtered_paragraphs:
    stemmed_p = []
    for w in p:
        stemmed_p.append(stemmer.stem(w))
    stemmed_paragraphs.append(stemmed_p)

stemmed_question = []
for w in filtered_question:
    stemmed_question.append(stemmer.stem(w))

# create a list of tokens for the paragraphs and question
tokens_p = []
tokens_q = []
for p in stemmed_paragraphs:
    tokens_p.append(p)
tokens_q.extend(stemmed_question)
print(tokens_q)

C:\Users\samad\AppData\Local\Temp\ipykernel_14984\2845631101.py:19: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


['purpos', 'preprocess', 'text', 'data']


In [4]:
path = 'train_data.csv'
data = pd.read_csv(path)

C:\Users\samad\AppData\Local\Temp\ipykernel_8952\2320214636.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [5]:
# TQ = ''
# for tok in tokens_q[0]:
#     TQ += tok + ' '
questions = []
for q in data['Question']:
    if q not in questions:
          questions.append(q)
themes = []
for t in data['Theme']:
    if t not in themes:
        themes.append(t)

In [7]:
Paragraphs = []
for para in data['Paragraph']:
    if para not in Paragraphs:
        Paragraphs.append(para)

In [22]:
P = data['Paragraph'][data['Theme'] == themes[0]]

In [23]:
Paras = []
for para in P:
    if para not in Paras:
        Paras.append(para)
Paras.append(questions[0])

Approach 3

In [24]:
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
Tokenizer = AutoTokenizer.from_pretrained(model_name)
Model = AutoModel.from_pretrained(model_name)
tokens = {'input_ids' : [], 'attention_mask' : []}
for el in Paras:
    new_tokens = Tokenizer.encode_plus(el, max_length = 128,
                                       truncation = True, padding = 'max_length',
                                       return_tensors = 'pt') 
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
outputs = Model(**tokens)
embeddings = outputs.last_hidden_state
attention = tokens['attention_mask']
mask = attention.unsqueeze(-1).expand(embeddings.shape).float()
mask_embeddings = embeddings*mask
summed = torch.sum(mask_embeddings, 1)
counts = torch.clamp(mask.sum(1), min = 1e-9)
mean_pooled = (summed/counts).detach().numpy()


In [28]:
similarity = cosine_similarity([mean_pooled[-1]], mean_pooled[:len(mean_pooled)-1])
similarity[0].tolist().index(max(similarity[0]))
similarity

array([[0.6226311 , 0.62663156, 0.4643542 , 0.6044539 , 0.5350107 ,
        0.5302644 , 0.4876517 , 0.47589162, 0.5421442 , 0.42167842,
        0.52171105, 0.64418894, 0.47883362, 0.52977973, 0.5128255 ,
        0.5041628 , 0.53310674, 0.59048355, 0.52646327, 0.47564873,
        0.5249543 , 0.5127542 , 0.5064064 , 0.54416806, 0.5075203 ,
        0.52174455, 0.5882031 , 0.47375882, 0.53860724, 0.45436174,
        0.5575513 , 0.4690459 , 0.42037868, 0.41298923, 0.4877195 ,
        0.5286655 , 0.56423485, 0.5745464 , 0.60030746, 0.59218776,
        0.42867064, 0.5934118 , 0.44822526, 0.6509285 , 0.4580393 ,
        0.462371  , 0.52888644, 0.47936013, 0.47673216, 0.46733046,
        0.5968343 , 0.61360645, 0.40107253, 0.40395582, 0.49821523,
        0.4034803 , 0.5708873 , 0.506723  , 0.48446858, 0.49564862,
        0.5106265 , 0.47472575, 0.4982315 , 0.38249168, 0.48430756,
        0.45806777]], dtype=float32)

In [30]:
Paras[-1]

"When did Beyonce leave Destiny's Child and become a solo singer?"

In [27]:
Paras[43]

'Described as being "sexy, seductive and provocative" when performing on stage, Beyoncé has said that she originally created the alter ego "Sasha Fierce" to keep that stage persona separate from who she really is. She described Sasha as being "too aggressive, too strong, too sassy [and] too sexy", stating, "I\'m not like her in real life at all." Sasha was conceived during the making of "Crazy in Love", and Beyoncé introduced her with the release of her 2008 album I Am... Sasha Fierce. In February 2010, she announced in an interview with Allure magazine that she was comfortable enough with herself to no longer need Sasha Fierce. However, Beyoncé announced in May 2012 that she would bring her back for her Revel Presents: Beyoncé Live shows later that month.'

Approach 2

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def predict_answer(question, paragraphs, theme):
    # Filter the paragraphs to only include those with the correct theme
    theme_paragraphs = [p['text'] for p in paragraphs if p['theme'] == theme]
    # cummulate = ' '.join(theme_paragraphs)
    # for trial in theme_paragraphs:
    #     encoding = tokenizer.encode_plus(text=question,text_pair= trial)
    #     inputs = encoding['input_ids']  #Token embeddings
    #     sentence_embedding = encoding['token_type_ids']  #Segment embeddings
    #     tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens
    #     start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
    #     start_index = torch.argmax(start_scores)
    #     end_index = torch.argmax(end_scores)
    #     final_answer = ' '.join(tokens[end_index:start_index+1])
    #     print(final_answer)
    # Create a TfidfVectorizer to convert the paragraphs into numerical vectors
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(theme_paragraphs)
    
    # Convert the question into a numerical vector
    question_vector = vectorizer.transform([question])
    
    # Calculate the cosine similarity between the question and each paragraph
    similarity = cosine_similarity(question_vector, X)
    
    # Find the paragraph with the highest similarity score
    max_similarity_index = similarity.argmax()
    max_similarity_score = similarity[0, max_similarity_index]
    
    # If the maximum similarity score is above a certain threshold, return the paragraph
    if max_similarity_score > 0.12:
        return theme_paragraphs[max_similarity_index]
    else:
        return None


In [29]:
cumm_para = ''
for c in Paragraphs:
    cumm_para += c+' '

In [31]:
from transformers import BertModel, BertTokenizer

# Initialize the BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased', return_dict=False)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [32]:
question = questions[0]
encoding = tokenizer.encode_plus(text=question,text_pair= cumm_para)
inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens


Token indices sequence length is longer than the specified maximum sequence length for this model (2382008 > 512). Running this sequence through the model will result in indexing errors


In [33]:
import torch
start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
start_scores


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 7317528576 bytes.

In [ ]:
# question = questions[0]
# paragraphs = [
#     {'text': 'Paris is the capital of France', 'theme': 'Geography'},
#     {'text': 'Lyon is a city in France', 'theme': 'Geography'},
#     {'text': 'The Pythagorean theorem states that a^2 + b^2 = c^2', 'theme': 'Mathematics'},
# ]

# # Predict the answer to the question
# answer = predict_answer(question, paragraphs, 'Geography')

# # Print the answer
# if answer:
#     print(answer)
# else:
#     print("No suitable paragraph found")

In [9]:
PT = []
Themes = []
s = 0
for element in Paragraphs:
    
    Themes.append(data['Theme'][data.loc[data['Paragraph'] == element]['Theme'].index[0]])


In [161]:
Solution = []
for element in questions:
    Solution.append(data['Answer_text'][data.loc[data['Question'] == element]['Answer_text'].index[0]])

In [79]:
data['Theme'][data.loc[data['Paragraph'] == Paragraphs[100]]['Theme'].index[0]]

'Frédéric_Chopin'

In [10]:
for i in range(len(Themes)):
    PT.append({'text': Paragraphs[i], 'theme': Themes[i]})

In [54]:
question = questions[0]
answer = predict_answer(question, PT, 'Beyoncé')
if answer:
    print(answer)
else:
    print("No suitable paragraph found")

2003



1990
1995
1999







[SEP]
[SEP]
2010

2011
2010

2013


2015


2011

2009

2015


[SEP]
90s

1991
2006
2006

2006
[SEP] beyonce
2008
2000s
2012
2002
[SEP] the bey hive is the name given to beyonce
2011
2007 , that he thinks race plays a role in many of these criticisms , saying white celebrities who dress similarly do not attract as many comments . in 2008

2012
2013
2009 , the observer named her the artist of the decade and billboard named her the top female artist and top radio songs artist of the decade . in 2010 , billboard named her in their " top 50 r & b / hip - hop artists of the past 25 years " list at number 15 . in 2012 vh1 ranked her third on their list of the " 100 greatest women in music " . beyonce was the first female artist to be honored with the international artist award at the american music awards . she has also received the legend award at the 2008
2010
2002




2005
2005

2005
2010



In [12]:
question

"Who managed the Destiny's Child group?"

In [13]:
answer

'The group changed their name to Destiny\'s Child in 1996, based upon a passage in the Book of Isaiah. In 1997, Destiny\'s Child released their major label debut song "Killing Time" on the soundtrack to the 1997 film, Men in Black. The following year, the group released their self-titled debut album, scoring their first major hit "No, No, No". The album established the group as a viable act in the music industry, with moderate sales and winning the group three Soul Train Lady of Soul Awards for Best R&B/Soul Album of the Year, Best R&B/Soul or Rap New Artist, and Best R&B/Soul Single for "No, No, No". The group released their multi-platinum second album The Writing\'s on the Wall in 1999. The record features some of the group\'s most widely known songs such as "Bills, Bills, Bills", the group\'s first number-one single, "Jumpin\' Jumpin\'" and "Say My Name", which became their most successful song at the time, and would remain one of their signature songs. "Say My Name" won the Best R&

In [14]:
from transformers import BertForQuestionAnswering, BertTokenizer

# Initialize the BERT model and tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad', return_dict=False)
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [15]:
encoding = tokenizer.encode_plus(text=question,text_pair= answer)
inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens


In [16]:
import torch

In [17]:
start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
start_scores

tensor([[-0.7706, -7.2730, -8.2646, -8.3917, -8.0885, -9.0185, -9.0886, -9.1559,
         -8.9324, -9.2371, -0.7706, -4.5914, -6.5819, -5.6477, -7.3688, -6.7837,
         -7.3916, -5.1371, -8.8219, -8.1138, -7.9400, -6.7756, -3.4717, -7.8041,
         -5.5063, -8.5547, -7.1756, -6.4896, -8.2490, -6.7955, -5.5777, -8.5795,
         -6.1490, -0.7713, -5.4493, -4.0023, -8.2799, -5.1208, -8.6579, -8.3759,
         -8.2101, -6.9559, -7.4337, -4.7180, -7.5110, -6.5353, -7.0225, -6.7830,
         -6.1602, -7.9648, -7.9754, -7.1390, -7.1975, -4.8113, -8.5358, -7.6213,
         -4.4523, -6.4658, -8.1768, -5.5290, -8.3147, -7.4403, -8.0779, -6.0659,
         -7.4882, -7.7166, -8.3190, -6.7098, -7.5801, -7.5903, -7.5324, -6.8471,
         -8.8198, -8.1568, -6.1447, -6.8109, -8.0516, -6.6115, -8.0142, -7.3577,
         -7.6770, -7.6329, -7.4716, -7.0594, -8.9132, -8.7029, -9.2535, -8.5108,
         -8.6886, -7.8206, -6.6940, -7.5430, -6.9247, -7.7752, -8.5381, -8.5796,
         -8.3026, -7.6781, -

In [18]:
start_index = torch.argmax(start_scores)

end_index = torch.argmax(end_scores)

final_answer = ''.join(tokens[end_index:start_index+1])

In [19]:
start_index

tensor(10)

In [20]:
end_index

tensor(10)

In [21]:
final_answer

'[SEP]'

In [239]:
len(answer)

1146